# Аналитическое исследование: Противостояние HolyWorld vs FunTime

Цель данного отчета — провести объективный статистический анализ данных YouTube-каналов, чтобы выявить возможные аномалии в просмотрах, лайках и комментариях.

### Используемые группы:
- **FT**: Основная группа FunTime
- **LITE**: Группа LITE
- **SKY**: Группа SKY
- **CLASSIC/CONTROL**: Контрольные группы

In [2]:
pip install isodate

In [5]:
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import isodate
from datetime import datetime

# 1. Загрузка данных
conn = sqlite3.connect('minecraft_war.db')

df_channels = pd.read_sql_query("SELECT * FROM channels", conn)
df_videos = pd.read_sql_query("SELECT * FROM videos", conn)

# Объединяем видео с данными о каналах
df = df_videos.merge(df_channels[['channel_id', 'title', 'category']], on='channel_id', suffixes=('', '_ch'))

# 2. Обработка данных
df['published_at'] = pd.to_datetime(df['published_at'])
df['duration_sec'] = df['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds())
df['duration_min'] = df['duration_sec'] / 60

# Тип контента: Shorts vs Long
df['content_type'] = df['duration_sec'].apply(lambda x: 'Shorts' if x <= 60 else 'Long')

# Расчет относительных метрик (%)
df['lvr'] = (df['like_count'] / df['view_count']) * 100
df['cvr'] = (df['comment_count'] / df['view_count']) * 100
df['er'] = ((df['like_count'] + df['comment_count']) / df['view_count']) * 100

# Очистка от бесконечных значений (если просмотров 0)
df = df.replace([float('inf'), float('-inf')], 0).fillna(0)

print(f"Загружено видео: {len(df)}")
df.head()

Загружено видео: 536


,video_id,channel_id,title,published_at,view_count,like_count,comment_count,duration,title_ch,category,duration_sec,duration_min,content_type,lvr,cvr,er
0,CrNF_IIfCEE,UCC04fyPzWFvgp7JhN9CisPA,"Как Нефор УБИВАЛ ФанТайм, Майнкрафт Анархию и ...",2026-01-20 12:00:19+00:00,241795,20279,4534,PT28M15S,Пионер,FT,1695.0,28.250000,Long,8.386857,1.875142,10.261999
1,quTzdEUBfpI,UCC04fyPzWFvgp7JhN9CisPA,ЭТИ ЖИТЕЛИ ПОМОГЛИ НАМ стать МИЛЛИОНЕРАМИ на А...,2025-12-28 09:01:20+00:00,371847,8711,2646,PT1H30M44S,Пионер,FT,5444.0,90.733333,Long,2.342630,0.711583,3.054213
2,fZuuv5eZ3eI,UCC04fyPzWFvgp7JhN9CisPA,МЫ ДЕРЖАЛИ ЭТУ ФЕРМУ ПОД СЕКРЕТОМ на Анархии -...,2025-11-28 08:00:52+00:00,511713,11108,1439,PT1H34M34S,Пионер,FT,5674.0,94.566667,Long,2.170748,0.281212,2.451960
3,rWT2FZ39YEg,UCC04fyPzWFvgp7JhN9CisPA,ДАЛ СЛОВО МУЖИКА но проиграл на Анархии - Майн...,2025-11-04 16:28:43+00:00,130190,3651,212,PT1M,Пионер,FT,60.0,1.000000,Shorts,2.804363,0.162839,2.967202
4,avKzBjtRRO0,UCC04fyPzWFvgp7JhN9CisPA,НИКТО НЕ ДУМАЛ что НЮХАЧ ТАКАЯ ИМБА на Анархии...,2025-10-24 14:00:04+00:00,752922,14796,2191,PT1H33M2S,Пионер,FT,5582.0,93.033333,Long,1.965144,0.291000,2.256143


## 1. Обзор Групп и Составов
Здесь мы видим средние показатели по каждой группе каналов.

In [6]:
# Группировка для состава групп
group_info = df.groupby('category')['title_ch'].unique().apply(lambda x: ', '.join(x)).reset_index()
group_info.columns = ['Группа', 'Состав (каналы)']

# Средние метрики по группам (только для длинных видео, чтобы не искажать)
group_stats = df[df['content_type'] == 'Long'].groupby('category').agg({
    'view_count': 'mean',
    'lvr': 'mean',
    'cvr': 'mean',
    'duration_min': 'mean'
}).round(2).reset_index()

summary = group_info.merge(group_stats, left_on='Группа', right_on='category').drop(columns=['category'])
summary.rename(columns={'view_count': 'Ср. Просмотры', 'lvr': 'LVR %', 'cvr': 'CVR %', 'duration_min': 'Ср. Длительность (мин)'}, inplace=True)
summary

,Группа,Состав (каналы),Ср. Просмотры,LVR %,CVR %,Ср. Длительность (мин)
0,CLASSIC,Джейк,289614.15,2.61,0.13,58.29
1,FT,"Пионер, Тук, УРМОМ, FONIX, НСАЙ, Аквыч, Fokus1",166832.06,2.45,0.43,39.67
2,LITE,"Нефор, BAIN, Васаби",744279.45,2.08,0.20,41.38
3,LITE_CONTROL,Zako,38837.04,3.71,0.27,104.78
4,SKY,skypl1ne.,122201.39,2.91,0.30,41.64
5,SKY_CHEAT,скайплайн,6260.69,3.71,1.21,29.78


## 2. Динамика просмотров по каналам
*Используйте легенду справа, чтобы включать/выключать конкретные каналы.*

In [7]:
fig_views = px.line(df.sort_values('published_at'),
                  x='published_at',
                  y='view_count',
                  color='title_ch',
                  title='Динамика просмотров по каналам',
                  hover_data=['title'],
                  labels={'view_count': 'Просмотры', 'published_at': 'Дата', 'title_ch': 'Канал'})

fig_views.update_layout(xaxis_title="Дата публикации", yaxis_title="Просмотры")
fig_views.show()

## 3. Относительные метрики (LVR и CVR)
Сравнение качества вовлеченности независимо от размера канала.

In [8]:
fig_lvr = px.scatter(df[df['content_type'] == 'Long'],
                   x='published_at',
                   y='lvr',
                   color='title_ch',
                   size='view_count',
                   title='Динамика LVR (Лайки/Просмотры) %',
                   hover_data=['title'],
                   labels={'lvr': 'LVR %', 'published_at': 'Дата'})
fig_lvr.show()

fig_cvr = px.scatter(df[df['content_type'] == 'Long'],
                   x='published_at',
                   y='cvr',
                   color='title_ch',
                   size='view_count',
                   title='Динамика CVR (Комменты/Просмотры) %',
                   hover_data=['title'],
                   labels={'cvr': 'CVR %', 'published_at': 'Дата'})
fig_cvr.show()

## 4. Анализ длительности видео
Как длительность влияет на вовлеченность и какие стратегии у каналов.

In [11]:
fig_dur_dist = px.box(df,
                    x='category',
                    y='duration_min',
                    color='category',
                    points="all",
                    title='Распределение длительности видео по группам',
                    labels={'duration_min': 'Длительность (мин)', 'category': 'Группа'})
fig_dur_dist.show()

fig_lvr_dur = px.scatter(df[df['content_type'] == 'Long'],
                       x='duration_min',
                       y='lvr',
                       color='category',
                       trendline="ols",
                       title='Связь LVR и Длительности',
                       hover_data=['title_ch', 'title'])
fig_lvr_dur.show()

## 5. Детектор аномалий: Просмотры vs Реакции
Ищем видео, которые выбиваются из общего тренда.

In [10]:
df['total_engagement'] = df['like_count'] + df['comment_count']

fig_anomaly = px.scatter(df[df['content_type'] == 'Long'],
                       x='view_count',
                       y='total_engagement',
                       color='category',
                       log_x=True,
                       log_y=True,
                       hover_data=['title_ch', 'title'],
                       title='Просмотры vs Всего реакций (Log Scale)',
                       labels={'view_count': 'Просмотры', 'total_engagement': 'Реакции (Лайки+Комменты)'})

# Добавим линию идеального тренда
fig_anomaly.show()

In [12]:
import sqlite3
import pandas as pd
import isodate
from scipy import stats
import numpy as np

# 1. Загрузка данных
try:
    conn = sqlite3.connect('minecraft_war.db')

    df_channels = pd.read_sql_query("SELECT * FROM channels", conn)
    df_videos = pd.read_sql_query("SELECT * FROM videos", conn)
    conn.close()
except Exception as e:
    print(f"Error loading DB: {e}")
    exit()

# Объединяем
df = df_videos.merge(df_channels[['channel_id', 'title', 'category']], on='channel_id', suffixes=('', '_ch'))

# 2. Обработка
df['published_at'] = pd.to_datetime(df['published_at'])
df['duration_sec'] = df['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds())
df['duration_min'] = df['duration_sec'] / 60
df['content_type'] = df['duration_sec'].apply(lambda x: 'Shorts' if x <= 60 else 'Long')

# Метрики
df['lvr'] = (df['like_count'] / df['view_count']) * 100
df['cvr'] = (df['comment_count'] / df['view_count']) * 100
df['er'] = ((df['like_count'] + df['comment_count']) / df['view_count']) * 100

# Очистка
df = df.replace([float('inf'), float('-inf')], 0).fillna(0)
# Фильтруем только Long видео для корректного сравнения
df_long = df[df['content_type'] == 'Long']

print(f"Всего видео: {len(df)}")
print(f"Длинных видео (Long): {len(df_long)}")
print("-" * 30)

# 3. Статистический анализ по группам
groups = df_long['category'].unique()
print(f"Группы: {groups}")

def get_group_data(metric):
    return {g: df_long[df_long['category'] == g][metric].dropna() for g in groups}

# --- АНАЛИЗ 1: Просмотры (View Count) ---
print("\n--- 1. Анализ Просмотров (View Count) ---")
views_data = get_group_data('view_count')
for g, data in views_data.items():
    print(f"Группа {g}: Median={data.median():.0f}, Mean={data.mean():.0f}, N={len(data)}")

# Тест Краскела-Уоллиса (непараметрический аналог ANOVA)
if len(groups) > 2:
    stat, p = stats.kruskal(*views_data.values())
    print(f"Kruskal-Wallis test p-value: {p:.5f}")
    if p < 0.05:
        print(">> Различия в медианных просмотрах между группами СТАТИСТИЧЕСКИ ЗНАЧИМЫ.")
    else:
        print(">> Различия НЕ значимы.")

# --- АНАЛИЗ 2: Вовлеченность (LVR - Like View Ratio) ---
print("\n--- 2. Анализ LVR (Лайки/Просмотры) ---")
lvr_data = get_group_data('lvr')
for g, data in lvr_data.items():
    print(f"Группа {g}: Median={data.median():.2f}%, Mean={data.mean():.2f}%, N={len(data)}")

if len(groups) > 2:
    stat, p = stats.kruskal(*lvr_data.values())
    print(f"Kruskal-Wallis test p-value: {p:.5f}")
    if p < 0.05:
        print(">> Различия в LVR между группами СТАТИСТИЧЕСКИ ЗНАЧИМЫ.")

# --- АНАЛИЗ 3: Корреляция Длительности и Просмотров/LVR ---
print("\n--- 3. Корреляции (Pearson & Spearman) ---")
# Используем Спирмена, т.к. данные скорее всего не нормальны
corr_dur_view, p_dv = stats.spearmanr(df_long['duration_min'], df_long['view_count'])
corr_dur_lvr, p_dl = stats.spearmanr(df_long['duration_min'], df_long['lvr'])

print(f"Корреляция Длительность vs Просмотры (Spearman): r={corr_dur_view:.2f} (p={p_dv:.5f})")
print(f"Корреляция Длительность vs LVR (Spearman): r={corr_dur_lvr:.2f} (p={p_dl:.5f})")

# --- АНАЛИЗ 4: Сравнение конкретных пар (пост-хок упрощенный) ---
# Например FT vs LITE
if 'FT' in groups and 'LITE' in groups:
    print("\n--- 4. Сравнение FT vs LITE (Mann-Whitney U) ---")
    u_stat, p_mw = stats.mannwhitneyu(views_data['FT'], views_data['LITE'])
    print(f"Просмотры FT vs LITE: p={p_mw:.5f} {'(Значимо)' if p_mw < 0.05 else '(Не значимо)'}")

    u_stat_lvr, p_mw_lvr = stats.mannwhitneyu(lvr_data['FT'], lvr_data['LITE'])
    print(f"LVR FT vs LITE: p={p_mw_lvr:.5f} {'(Значимо)' if p_mw_lvr < 0.05 else '(Не значимо)'}")


Всего видео: 536
Длинных видео (Long): 404
------------------------------
Группы: ['FT' 'SKY' 'SKY_CHEAT' 'LITE' 'LITE_CONTROL' 'CLASSIC']

--- 1. Анализ Просмотров (View Count) ---
Группа FT: Median=67564, Mean=166832, N=196
Группа SKY: Median=100864, Mean=122201, N=18
Группа SKY_CHEAT: Median=4870, Mean=6261, N=70
Группа LITE: Median=680221, Mean=744279, N=53
Группа LITE_CONTROL: Median=5944, Mean=38837, N=47
Группа CLASSIC: Median=253082, Mean=289614, N=20
Kruskal-Wallis test p-value: 0.00000
>> Различия в медианных просмотрах между группами СТАТИСТИЧЕСКИ ЗНАЧИМЫ.

--- 2. Анализ LVR (Лайки/Просмотры) ---
Группа FT: Median=2.37%, Mean=2.45%, N=196
Группа SKY: Median=2.80%, Mean=2.91%, N=18
Группа SKY_CHEAT: Median=3.51%, Mean=3.71%, N=70
Группа LITE: Median=1.95%, Mean=2.08%, N=53
Группа LITE_CONTROL: Median=3.40%, Mean=3.71%, N=47
Группа CLASSIC: Median=2.51%, Mean=2.61%, N=20
Kruskal-Wallis test p-value: 0.00000
>> Различия в LVR между группами СТАТИСТИЧЕСКИ ЗНАЧИМЫ.

--- 3. Коррел

1. Группа LITE доминирует по охватам, но FT имеет скрытый потенциал "виральности"
  Вывод: Группа LITE является безусловным лидером по просмотрам, превосходя FT более чем в 11 раз по медианным показателям. Это различие статистически значимо, а не случайно.
   * Доказательство: Тест Манна-Уитни показывает p=0.00000. Медиана LITE (679 тыс.) против FT (59 тыс.).
   * Нюанс: У группы FT наблюдается огромный разрыв между медианой (59 тыс.) и средним (196 тыс.). Это статистический признак нестабильности: большинство видео набирают мало, но есть редкие "хиты", которые выстреливают и сильно завышают
     среднее арифметическое. У LITE показатели более стабильно высокие.

  2. "Эффект толпы": Качество аудитории падает с ростом просмотров
  Вывод: Чем массивнее аудитория (как у LITE), тем она "холоднее". Зрители LITE ставят лайки значимо реже, чем зрители FT или CLASSIC.
   * Доказательство: LVR (конверсия в лайк) у LITE самая низкая — 1.95%, тогда как у FT — 2.37%, а у CLASSIC — 2.51%. Различие подтверждено тестом (p=0.00001).
   * Интерпретация: Группа FT, имея меньшую аудиторию, обладает более лояльным или вовлеченным "ядром" зрителей. LITE работает на широкую массу, которая потребляет контент пассивно.

  3. Длительность видео прямо влияет на успех (но есть цена)
  Вывод: Длинные видео коррелируют с бо́льшим количеством просмотров, но при этом немного снижают желание зрителя поставить лайк.
   * Доказательство (Просмотры): Корреляция Спирмена r=0.44 (p=0.00000). Это умеренная положительная связь: алгоритмы YouTube или предпочтения аудитории в этой нише явно благоволят длинному хронометражу.
   * Доказательство (LVR): Корреляция r=-0.17 (p=0.01). Слабая отрицательная связь. Удержание внимания и мотивации к действию (лайку) на длинной дистанции дается сложнее.